# Do inference in paralllel

In [ ]:
import os
import glob
import pickle
import time
import requests
from concurrent.futures import ThreadPoolExecutor

In [ ]:
from get_inference_service_url import get_inference_service_url
infer_url = get_inference_service_url()

In [ ]:
# how many requests to send
MAX_REQUESTS=50

In [ ]:
# Function to send a single REST request
def rest_request(data):
    json_data = {
        "inputs": [
            {
                "name": "pixel_values",
                "shape": data.shape,
                "datatype": "FP32",
                "data": data.tolist()
            }
        ]
    }

    start_time = time.time()
    response = requests.post(infer_url, json=json_data, verify=True)
    end_time = time.time()

    if response.status_code != 200:
        print(f"Request failed with status code: {response.status_code}")

    response_time = end_time - start_time
    print(f"Response time: {response_time:.6f} seconds")

    return response

In [ ]:
pickled_file = "pickled/pickled_tensors.pkl"

with open(pickled_file, 'rb') as file:
    mytensors = pickle.load(file)

# Limit the output to the first 50 .pkl files
limited_tensors = list(mytensors.values())[:MAX_REQUESTS]


def inference(data):
    pixel_values = data["pixel_values"]
    rest_request(pixel_values)

start_time = time.time()

# Process files in parallel
with ThreadPoolExecutor(max_workers=8) as executor:
    executor.map(inference, limited_tensors)

end_time = time.time()

# Print summary
elapsed_time = end_time - start_time
print(f"Total files processed: {len(limited_tensors)}")
print(f"Time taken: {elapsed_time:.2f} seconds")
